In [1]:
# change the working directory to point to the root directory
import os

os.chdir("../")

In [2]:
# imports
import numpy as np
import pandas as pd

In [3]:
# imports for plots
from plotly import graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "plotly_white"

In [4]:
# structure of dict
{ 
    "univariate": {
        "cumulative_PRECTOT": { "nb_clusters": 0, "score": 0, "clusters": {} },
        "cumulative_PRECTOT": { "nb_clusters": 0, "score": 0, "clusters": {} },
        "cumulative_PRECTOT": { "nb_clusters": 0, "score": 0, "clusters": {} },
        "cumulative_PRECTOT": { "nb_clusters": 0, "score": 0, "clusters": {} },
    },
    "multivariate": { "nb_clusters": 0, "score": 0, "clusters": {} }
}

{'univariate': {'cumulative_PRECTOT': {'nb_clusters': 0,
   'score': 0,
   'clusters': {}}},
 'multivariate': {'nb_clusters': 0, 'score': 0, 'clusters': {}}}

In [5]:
# load data
data_region = pd.read_csv("raw_data/weather_regions/33.79102828115297;-3.67179828879979;Taza-Al Hoceima-Taounate;Guercif.csv")

In [6]:
# imports and general params
from py_scripts.clustering import isolate_data_col
from py_scripts.clustering import reduce_merge_multivariate_data, multivariate_clustering, univariate_clustering
from py_scripts.clustering import get_best_number_clusters

selected_cols = ["cumulative_GDD", "cumulative_PRECTOT", "cumulative_WS2M", "cumulative_RH2M"]

In [7]:
def generate_info_region(data_region):
    # univariate
    dict_uni_var = {}
    for col in selected_cols:
        # construct data
        data_var = isolate_data_col(data=data_region, col_name=col)

        # where to store info of var
        dict_var = { "clusters": {}}

        # nb clusters and corresponding score
        dict_var["nb_clusters"], dict_var["score"] = max(list(get_best_number_clusters(data_var).items()), key=lambda x: x[1])

        # cluster data (use entire data region)
        dict_var["clusters"] = univariate_clustering(data_region, col_name=col, nb_clusters=dict_var["nb_clusters"])

        # save result clustering of col
        dict_uni_var[col] = dict_var

    # multivariate
    dict_multi_var = {}
    reduced_data = reduce_merge_multivariate_data({ col: isolate_data_col(data_region, col) for col in selected_cols})

    # ideal nb clusters and score
    dict_multi_var["nb_clusters"], dict_multi_var["score"] = max(list(get_best_number_clusters(reduced_data).items()), key=lambda x: x[1])

    # multi clustering clustering
    dict_multi_var["clusters"] = multivariate_clustering(data_region, nb_clusters=dict_multi_var["nb_clusters"])

    return { "univariate": dict_uni_var, "multivariate": dict_multi_var }

In [9]:
# get dict info from data region
dict_all_infos = generate_info_region(data_region)

In [11]:
dict_all_infos["multivariate"]

{'nb_clusters': 2,
 'score': 36.13377939286226,
 'clusters': {0: Int64Index([1991, 1994, 1996, 2001, 2004, 2006, 2009, 2010, 2011, 2012, 2013,
              2015, 2018],
             dtype='int64'),
  1: Int64Index([1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1992, 1993,
              1995, 1997, 1998, 1999, 2000, 2002, 2003, 2005, 2007, 2008, 2014,
              2016, 2017, 2019, 2020],
             dtype='int64')}}

In [14]:
# Save
np.save('processed_data/test_region.npy', dict_all_infos)

# Load
read_dictionary = np.load('processed_data/test_region.npy', allow_pickle='TRUE').item()

In [27]:
max([(key, val['nb_clusters'], val["score"]) for key, val in dict_all_infos["univariate"].items()], key=lambda x: x[2])

('cumulative_RH2M', 5, 54.43284969426086)

In [24]:
# # generate infos of all regions

# dir_path = "raw_data/weather_regions/"
# for file in os.listdir(dir_path):
#     # how dict will be named: remove extension (.csv) from file
#     filename = file[:-4]

#     # load data region
#     data_region = pd.read_csv(dir_path + file)

#     # get all infos
#     dict_all_infos = generate_info_region(data_region)

#     # save dict
#     np.save(f"processed_data/individual_regions/{filename}.npy", dict_all_infos)